In [1]:
from main import init_detectron
import os
from tqdm.notebook import tqdm
from random import sample
import cv2
import numpy as np
from itertools import compress
import torchvision
from torch.utils.data import DataLoader, random_split
from stacker_net.train import MyDataset, train, train_autoencoder
from stacker_net.net import StackerNet
from torch import nn
import torch
import json

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [2]:
data = []
for root, dirs, files in os.walk('data/data_256'):
    if files:
        data.append((root, files))

In [3]:
predictor, coi = init_detectron()

In [4]:
selection_filename = 'data/selected_images.json'
force_overwrite = False

if not os.path.isfile(selection_filename) or force_overwrite:
    global ratios, data2, paths
    ratios = []
    for category in tqdm(data):
        samples = sample(category[1], 25)
        total = len(samples)
        positives = 0
        for img_name in samples:
            img = cv2.imread(os.path.join(category[0], img_name))
            outputs = predictor(img)
            if coi.intersection(outputs['instances'].pred_classes.cpu().numpy()):
                positives += 1
        ratios.append(positives / total)
    data2 = list(compress(data, np.array(ratios) >= 0.96))
    paths = [os.path.join(category[0], file) for category in data2 for file in category[1]]
    with open(selection_filename, 'w') as f:
        json.dump(paths, f)
else:
    with open(selection_filename, 'r') as f:
        paths = json.load(f)

In [5]:
transforms = torchvision.transforms.Compose([
    torchvision.transforms.PILToTensor(),
    torchvision.transforms.ConvertImageDtype(torch.float32)
])
ds = MyDataset('data/data_256', paths, transform=transforms)

In [6]:
BATCH_SIZE = 128

train_len = int(len(ds) * 0.8)
val_len = len(ds) - train_len
train_ds, val_ds = random_split(ds, (train_len, val_len))
train_dl = DataLoader(train_ds, BATCH_SIZE, shuffle=True, pin_memory=True)
val_dl = DataLoader(val_ds, BATCH_SIZE, pin_memory=True)

In [7]:
len(ds)

369992

In [8]:
model = StackerNet().to(device)
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())

train_autoencoder(model, loss_fn, optimizer, train_dl, val_dl, pred_len=32, device=device, print_metrics=True, epochs=10)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/2313 [00:00<?, ?it/s]

Epoch 0: train loss = 176.373, val loss = 84.671


  0%|          | 0/2313 [00:00<?, ?it/s]

Epoch 1: train loss = 84.672, val loss = 84.671


  0%|          | 0/2313 [00:00<?, ?it/s]

Epoch 2: train loss = 84.669, val loss = 84.668


  0%|          | 0/2313 [00:00<?, ?it/s]

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.

In [ ]:
# model = StackerNet().to(device)
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())

train(model, loss_fn, optimizer, train_dl, val_dl, pred_len=32, device=device, print_metrics=True, epochs=15)

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/2313 [00:00<?, ?it/s]

Epoch 0: train loss = 121.554, val loss = 121.711


  0%|          | 0/2313 [00:00<?, ?it/s]

Epoch 1: train loss = 121.554, val loss = 121.711


  0%|          | 0/2313 [00:00<?, ?it/s]

Epoch 2: train loss = 121.554, val loss = 121.711


  0%|          | 0/2313 [00:00<?, ?it/s]

KeyboardInterrupt: 